<a href="https://colab.research.google.com/github/vuhoang123/TEXT-CLASSIFICATION/blob/main/VNTC_TEXT_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd ../../..
!ls

In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/duyvuleo/VNTC.git

In [ ]:
%cd VNTC/Data/10Topics/Ver1.1
!apt install unrar

In [ ]:
!unrar x Test_Full.rar
!unrar x Train_Full.rar 
!ls

In [ ]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch transformers tqdm sklearn Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import torch
from typing import List, Tuple, Dict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
!ls

'Colab Notebooks'   VNTC


In [ ]:
%cd VNTC/Data/10Topics/Ver1.1
!ls

/content/drive/MyDrive/VNTC/Data/10Topics/Ver1.1
Stats.txt  Test_Full  Test_Full.rar  Train_Full  Train_Full.rar


In [ ]:
TRAIN_PATH = "Train_Full/"
TEST_PATH = "Test_Full/"

In [ ]:
category2id = {category: idx for idx, category in enumerate(os.listdir(TRAIN_PATH))}
category2id

{'Doi song': 0,
 'Khoa hoc': 1,
 'Kinh doanh': 2,
 'Phap luat': 3,
 'Suc khoe': 4,
 'The gioi': 5,
 'The thao': 6,
 'Van hoa': 7,
 'Vi tinh': 8,
 'Chinh tri Xa hoi': 9}

In [ ]:
data = []
def read_file(category, file_name):
    text_path = os.path.join(TRAIN_PATH, category, file_name)
    with open(text_path, "r", encoding = 'utf-16') as file:
        content = file.read()
    return (content, category2id[category])
print("TRAIN:")
for category in os.listdir(TRAIN_PATH):
    category_path = os.path.join(TRAIN_PATH, category)
    print(category_path)
    data.extend([ read_file(category, file_name) for file_name in os.listdir(category_path)])
test_data = []
print("TEST:")
for category in os.listdir(TEST_PATH):
    category_path = os.path.join(TRAIN_PATH, category)
    print(category_path)
    test_data.extend([ read_file(category, file_name) for file_name in os.listdir(category_path)])

TRAIN:
Train_Full/Doi song
Train_Full/Khoa hoc
Train_Full/Kinh doanh
Train_Full/Phap luat
Train_Full/Suc khoe
Train_Full/The gioi
Train_Full/The thao
Train_Full/Van hoa
Train_Full/Vi tinh
Train_Full/Chinh tri Xa hoi
TEST:
Train_Full/Khoa hoc
Train_Full/Phap luat
Train_Full/Suc khoe
Train_Full/The gioi
Train_Full/The thao
Train_Full/Vi tinh
Train_Full/Chinh tri Xa hoi
Train_Full/Doi song
Train_Full/Kinh doanh
Train_Full/Van hoa


In [ ]:
class VNTCDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [ ]:
train_data, valid_data = train_test_split(data, test_size = 0.1)

In [ ]:
vntc_train_data = VNTCDataset(train_data)
vntc_valid_data = VNTCDataset(valid_data)
vntc_test_data = VNTCDataset(test_data)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print("Model max_length: ", tokenizer.model_max_length)

Model max_length:  256


In [ ]:
def collate_fn(data: List[Tuple[str, int]]):
    texts = []
    labels = []
    for content, label in data:
        texts.append(content)
        labels.append(label)    
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    return input_ids, labels

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset = vntc_train_data, batch_size = 32, collate_fn = collate_fn, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(dataset = vntc_valid_data, batch_size = 32, collate_fn = collate_fn)
test_dataloader = torch.utils.data.DataLoader(dataset = vntc_test_data, batch_size = 32, collate_fn = collate_fn)

In [ ]:
for input_ids, labels in train_dataloader:
    print(input_ids, labels)
    break

tensor([[    0,  4150,    69,  ...,     1,     1,     1],
        [    0, 51899,  2548,  ...,   127,  2209,     2],
        [    0,   125,  3992,  ...,    51,  1278,     2],
        ...,
        [    0,  1070,   948,  ...,    10,    36,     2],
        [    0,  8927, 24350,  ...,  2002,    13,     2],
        [    0, 14831,  3520,  ...,   566,   128,     2]]) tensor([6, 2, 3, 8, 9, 9, 4, 8, 9, 3, 9, 0, 3, 4, 2, 6, 9, 6, 6, 9, 5, 8, 0, 7,
        9, 6, 5, 0, 6, 6, 4, 4])


MODEL

In [ ]:
class PhobertForNewsClassification(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels = num_classes)
        for param in self.phobert.roberta.parameters():
            param.requires_grad = False
    def forward(self, input_ids, labels):
        out = self.phobert(input_ids, labels = labels)
        probs = torch.softmax(out['logits'], dim = -1)
        preds = torch.argmax(probs, dim = -1).cpu().detach().numpy()
        out['preds'] = preds
        return out

In [ ]:
model = PhobertForNewsClassification(len(category2id))

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [ ]:
def get_num_parameters(module: torch.nn.Module) -> Dict[str, int]:
    """

    Get the number of trainable, fixed and total parameters of a pytorch module.

    :param module: pytorch module
    :return: dict containing number of parameters
    """
    trainable = 0
    fixed = 0
    for p in module.parameters():
        if p.requires_grad:
            trainable += p.numel()
        else:
            fixed += p.numel()
    return {"trainable": trainable, "fixed": fixed, "total": trainable + fixed}

In [ ]:
get_num_parameters(model)

{'trainable': 598282, 'fixed': 134407680, 'total': 135005962}

TRAIN

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
LR = 1e-4
EPOCH = 5
LOG_STEP = 100

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr= LR)

In [ ]:
print(f"Train Loader Step: {len(train_dataloader)}")
print(f"Valid Loader Step: {len(valid_dataloader)}")

Train Loader Step: 950
Valid Loader Step: 106


In [ ]:
model = model.to(device)

In [ ]:
losses = 0
for i in range(EPOCH):
    print(f"Epoch {i}:")
    model.train()
    for step, (input_ids, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        out = model(input_ids, labels)
        loss = out['loss']
        loss.backward()
        optimizer.step()
        losses += loss.detach().cpu().item()
        if step % LOG_STEP == 0 and step > 0:
            print(f"Step {step}, Avg Loss {loss}")
            losses = 0
    
    preds = []
    trues = []
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for step, (input_ids, labels) in enumerate(valid_dataloader):
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            out = model(input_ids, labels)
            loss = out['loss']
            valid_loss += loss.detach().cpu().item()
            preds.extend(out['preds'])
            trues.extend(labels.detach().cpu().numpy())
    print(classification_report(trues, preds, target_names = list(category2id.keys())))
    print(f"Valid loss: {valid_loss / len(valid_dataloader)}")
    torch.save(model.state_dict(), f"breakpoint_epoch{i}.pt")

Epoch 0:
Step 100, Avg Loss 1.0873827934265137
Step 200, Avg Loss 0.7736279964447021
Step 300, Avg Loss 0.6264583468437195
Step 400, Avg Loss 0.8315905928611755
Step 500, Avg Loss 0.6439021825790405
Step 600, Avg Loss 0.44682544469833374
Step 700, Avg Loss 0.6184277534484863
Step 800, Avg Loss 0.47307366132736206
Step 900, Avg Loss 0.6795957684516907
                  precision    recall  f1-score   support

        Doi song       0.79      0.75      0.77       342
        Khoa hoc       0.78      0.73      0.75       195
      Kinh doanh       0.84      0.74      0.79       264
       Phap luat       0.80      0.84      0.82       407
        Suc khoe       0.79      0.84      0.81       379
        The gioi       0.89      0.86      0.87       271
        The thao       0.98      0.96      0.97       510
         Van hoa       0.79      0.83      0.81       272
         Vi tinh       0.94      0.76      0.84       240
Chinh tri Xa hoi       0.67      0.77      0.72       496

       

In [ ]:
preds = []
trues = []
model.eval()
with torch.no_grad():
    for step, (input_ids, labels) in enumerate(test_dataloader):
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        out = model(input_ids, labels)
        loss = out['loss']
        preds.extend(out['preds'])
        trues.extend(labels.detach().cpu().numpy())
        
print(classification_report(trues, preds, target_names = list(category2id.keys())))

                  precision    recall  f1-score   support

        Doi song       0.87      0.77      0.81      3159
        Khoa hoc       0.83      0.73      0.78      1820
      Kinh doanh       0.80      0.78      0.79      2552
       Phap luat       0.76      0.85      0.80      3868
        Suc khoe       0.78      0.90      0.83      3384
        The gioi       0.91      0.85      0.88      2898
        The thao       0.98      0.96      0.97      5298
         Van hoa       0.89      0.86      0.88      3080
         Vi tinh       0.91      0.86      0.89      2481
Chinh tri Xa hoi       0.72      0.75      0.73      5219

        accuracy                           0.84     33759
       macro avg       0.84      0.83      0.84     33759
    weighted avg       0.84      0.84      0.84     33759



In [ ]:
torch.save(model.state_dict(), "model.pt")

In [ ]:
text = """
Barcelona đã hoàn tất các thủ tục chiêu mộ hậu vệ Inigo Martinez từ Athletic Bilbao theo dạng chuyển nhượng tự do. Hợp đồng sẽ chính thức được ký vào cuối mùa giải này. Hiện tại đôi bên đã xong hầu hết mọi thoả thuận cần thiết, Martinez cũng đã hoàn tất kiểm tra y tế.
- Chủ tịch Joan Laporta khẳng định Barcelona muốn đưa Lionel Messi trở lại, bất chấp sự cạnh tranh từ đội bóng nhà giàu Al-Hilal. Mặc dù vậy, Laporta cũng thừa nhận ông không thể đưa ra một lời đề nghị quá điên rồ.
- Thêm một thông tin liên quan tới Barcelona: CLB xác nhận rằng Jordi Cruyff sẽ rời vị trí GĐTT vào cuối tháng 6 tới. Trước đó, đội bóng xứ Catalunya cũng đang loay hoay tìm người thay thế vị trí giám đốc bóng đá của Mateu Alemany. Hiện có tin cho rằng, huyền thoại Deco sẽ đảm nhiệm vai trò của Cruyff để lại.
"""

In [ ]:
encoded_input = tokenizer.encode_plus(
    text,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
input_ids = encoded_input["input_ids"]
labels = None 

In [ ]:
input_ids = input_ids.to(device)
with torch.no_grad():
    output = model(input_ids, labels)
logits = output['logits']
probs = torch.softmax(logits, dim=-1)
pred_label_id = torch.argmax(probs, dim=-1).item()
print("Nhãn dự đoán:", list(category2id.keys())[list(category2id.values()).index(pred_label_id)])

Nhãn dự đoán: The thao
